# TensorFlow Chess engine

 ### Dataset


In [3]:
import os

files = [file for file in os.listdir("../Database") if file.endswith('.pgn')]

In [4]:
len(files)

110

In [5]:
from chess import pgn

def load_pgn(file_path):
    games = []
    with open(file_path, 'r') as file:
        while True:
            game = pgn.read_game(file)
            if game is None:
                break
            games.append(game)
    return games



In [6]:
from tqdm import tqdm

LIMIT_OF_FILES = min(len(files), 12)
games = []

i = 1

for file in tqdm(files):
    games.extend(load_pgn(f"../Database/{file}"))
    if (i >= LIMIT_OF_FILES):
        break

    i+=1



 10%|█         | 11/110 [00:04<00:41,  2.37it/s]


In [7]:
len(games)

1341

# Build & Train a neural network


In [8]:
import numpy as np
from chess import Board
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
import pickle

In [9]:
def board_to_matrix(board: Board):
    matrix = np.zeros((8, 8, 12))
    piece_map = board.piece_map()
    for square, piece in piece_map.items():
        row,col = divmod(square, 8)
        piece_type = piece.piece_type -1
        piece_color = 0 if piece.color else 6
        matrix[row, col, piece_type + piece_color] = 1
    return matrix

def create_input_for_nn(games):
    X=[]
    y = []
    for game in games:
        board = game.board()
        for move in game.mainline_moves():
            X.append(board_to_matrix(board))
            y.append(move.uci())
            board.push(move)
    return X,y

def encode_moves (moves):
    move_to_int = {move: i for i, move in enumerate(set(moves))}
    return [move_to_int[move] for move in moves], move_to_int

In [10]:
X,y = create_input_for_nn(games)
y, move_to_int = encode_moves(y)
y = to_categorical(y, num_classes=len(move_to_int))
X = np.array(X)

In [217]:


if os.path.exists('../model/chess_model.keras'):
    model = tf.keras.models.load_model('../model/tensorflow/chess_model_tensorflow.keras')
else:                                                                              
    model = Sequential([
        Conv2D(64, (3, 3), activation='relu', input_shape=(8, 8, 12)),
        Conv2D(128, (3, 3), activation='relu'),
        Flatten(),
        Dense(256, activation='relu'),
        Dense(len(move_to_int), activation='softmax')
    ])
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()
model.fit(X, y, epochs=50, batch_size=64, validation_split=0.1)
if not os.path.exists('../model'):
    os.makedirs('../model')
model.save('../model/tensorflowchess_model_tensorflow.keras')


# Save move_to_int dictionary
with open('../model/tensorflow/move_to_int.pkl', 'wb') as f:
    pickle.dump(move_to_int, f)



C:\Users\Ricar\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)               │ (None, 6, 6, 64)       │         6,976 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 4, 4, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │       524,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1815)           │       466,455 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,071,831 (4.09 MB)

 Trainable params: 1,071,831 (4.09 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
1606/1606 ━━━━━━━━━━━━━━━━━━━━ 45s 26ms/step - accuracy: 0.0376 - loss: 6.4072 - val_accuracy: 0.0840 - val_loss: 5.7880
Epoch 2/50
1606/1606 ━━━━━━━━━━━━━━━━━━━━ 52s 33ms/step - accuracy: 0.1091 - loss: 5.4403 - val_accuracy: 0.1118 - val_loss: 5.3597
Epoch 3/50
1606/1606 ━━━━━━━━━━━━━━━━━━━━ 65s 41ms/step - accuracy: 0.1516 - loss: 4.7291 - val_accuracy: 0.1221 - val_loss: 5.2191
Epoch 4/50
1606/1606 ━━━━━━━━━━━━━━━━━━━━ 135s 84ms/step - accuracy: 0.1898 - loss: 4.1433 - val_accuracy: 0.1196 - val_loss: 5.2739
Epoch 5/50
1606/1606 ━━━━━━━━━━━━━━━━━━━━ 74s 46ms/step - accuracy: 0.2307 - loss: 3.6752 - val_accuracy: 0.1283 - val_loss: 5.3621
Epoch 6/50
1606/1606 ━━━━━━━━━━━━━━━━━━━━ 58s 31ms/step - accuracy: 0.2768 - loss: 3.2839 - val_accuracy: 0.1225 - val_loss: 5.6159
Epoch 7/50
1606/1606 ━━━━━━━━━━━━━━━━━━━━ 49s 31ms/step - accuracy: 0.3210 - loss: 2.9614 - val_accuracy: 0.1271 - val_loss: 5.9100
Epoch 8/50
1606/1606 ━━━━━━━━━━━━━━━━━━━━ 48s 30ms/step - accuracy: 0.3678 

In [218]:
from tensorflow.keras.models import load_model
model = load_model('../model/tensorflow/chess_model_tensorflow.keras')

if not move_to_int:
    with open('../model/tensorflow/move_to_int.pkl', 'rb') as f:
        move_to_int = pickle.load(f)


In [219]:
int_to_move = dict(zip(move_to_int.values(), move_to_int.keys()))

def predict_move(board):
    board_matrix = board_to_matrix(board).reshape(1, 8, 8, 12)
    predictions = model.predict(board_matrix)[0]
    legal_moves = list(board.legal_moves)
    uci_legal = [move.uci() for move in legal_moves]
    move_index = np.argsort(predictions)[::-1]
    for move_index in move_index:
        move = int_to_move[move_index]
        if move in uci_legal:
            return move
    return None

In [220]:
board = Board()

In [221]:


print("Board before prediction: ")
board

# Predict move
while not board.is_game_over():
    move = predict_move(board)
    board.push_uci(move)

    print("Predicted move: ", move)
    print("Board after prediction: ")
    board
    
print(str(pgn.Game.from_board(board)))



Board before prediction: 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 478ms/step
Predicted move:  e2e4
Board after prediction: 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
Predicted move:  d7d5
Board after prediction: 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
Predicted move:  e4d5
Board after prediction: 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
Predicted move:  d8d5
Board after prediction: 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
Predicted move:  g1f3
Board after prediction: 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
Predicted move:  b8c6
Board after prediction: 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step
Predicted move:  f1e2
Board after prediction: 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
Predicted move:  c8g4
Board after prediction: 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step
Predicted move:  h2h3
Board after prediction: 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
Predicted move:  g4f3
Board after prediction: 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
Predicted move:  e2f3
Board after prediction: 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 174ms